In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
traindf = pd.read_csv('train_label.csv', dtype=str)

def append_ext(fn):
    return fn+".png"

traindf["ID"]=traindf["ID"].apply(append_ext)

In [14]:
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25, zoom_range=0.2,)
# datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25, )
        
train_generator = datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "train_image/",
    x_col = "ID",
    y_col = "Label",
    subset = "training",
    batch_size = batch_size,
    seed = 23,
    shuffle = True,
    class_mode = "categorical",
    target_size = (512,512)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "train_image/",
    x_col = "ID",
    y_col = "Label",
    subset = "validation",
    batch_size = batch_size,
    seed = 23,
    shuffle = True,
    class_mode = "categorical",
    target_size = (512,512)
)

Found 873 validated image filenames belonging to 3 classes.
Found 291 validated image filenames belonging to 3 classes.


In [15]:
test_generator = datagen.flow_from_directory(
    "test_image", 
    batch_size = batch_size, 
    class_mode=None, 
    shuffle=False,
    target_size = (512,512)
)

Found 292 images belonging to 1 classes.


In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

# dimensions of our images.
img_width, img_height = 512, 512

nb_train_samples = 873
nb_validation_samples = 291
epochs = 50 

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [18]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/50
27/27 [==============================] - 81s 3s/step - loss: 2.1318 - accuracy: 0.5446 - val_loss: 0.6444 - val_accuracy: 0.6771
Epoch 2/50
27/27 [==============================] - 95s 4s/step - loss: 0.6644 - accuracy: 0.6766 - val_loss: 0.5475 - val_accuracy: 0.7188
Epoch 3/50
27/27 [==============================] - 1957s 72s/step - loss: 0.5823 - accuracy: 0.7325 - val_loss: 0.5260 - val_accuracy: 0.7222
Epoch 4/50
27/27 [==============================] - 101s 4s/step - loss: 0.5592 - accuracy: 0.7170 - val_loss: 0.4686 - val_accuracy: 0.7882
Epoch 5/50
27/27 [==============================] - 679s 25s/step - loss: 0.5318 - accuracy: 0.7372 - val_loss: 0.4661 - val_accuracy: 0.7431
Epoch 6/50
27/27 [==============================] - 96s 4s/step - loss: 0.4849 - accuracy: 0.7622 - val_loss: 0.4129 - val_accuracy: 0.8125
Epoch 7/50
27/27 [==============================] - 96s 4s/step - loss: 0.4779 - accuracy: 0.7598 - val_loss: 0.4479 - val_accuracy: 0.7396
Epoch 8/50
27/

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 508, 508, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 508, 508, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
activation_6 (Activation)    (None, 252, 252, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 124, 124, 64)     

In [19]:
predicted = model.predict(
    test_generator,
    batch_size=batch_size,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

In [22]:
prediction = predicted.argmax(axis=1)

In [25]:
np.savetxt("predicted.csv", np.dstack((np.arange(prediction.size),prediction))[0],"%d,%d",header="ID,Label",comments="", )